In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/home/laurent/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import pickle
import datetime
#from mix_Network import *
from tonic.transforms import AERtoVector

dataset = 'nmnist'
path = f'../Records/{dataset}/models/'
timestr = '2021-02-16'

%mkdir -p ../Records
#%mkdir -p ../Records/EXP_03_NMNIST

nb_train = 5000
nb_test = 1000

ds = 50
nb_train_b = nb_train//ds
nb_test_b = nb_test//ds

In [3]:
sigma = None
pooling = False
homeinv = False
jitonic = [None,None] #[temporal, spatial]
jitter = False
tau = 5
nblay = 3
nbclust = 4
filt = 2
def netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter):
    if name=='hots':
        homeo = False
        homeotest = False
        krnlinit = 'first'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learning1by1()
    elif name=='homhots':
        homeo = True
        homeotest = True
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    elif name=='onlyonline':
        homeo = False
        homeotest = False
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    return hotshom, homeotest

In [3]:
#for name in ['onlyonline', 'homhots', 'hots']:
for name in ['homhots']:
    hotshom, homeotest = netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter)
    stream_train = hotshom.running(homeotest = homeotest, nb_digit=nb_train, train=True, LR=True)
    stream_test = hotshom.running(homeotest = homeotest, nb_digit=nb_test, train=False, LR=True)

2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_5000_[None, None]_homeo.pkl
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_1000_[None, None]_homeo.pkl


In [17]:

#for name in ['onlyonline', 'homhots', 'hots']:
for name in ['homhots']:
    hotshom, homeotest = netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter)
    stream_train = hotshom.running(homeotest = homeotest, nb_digit=nb_train, train=True, LR=True)
    stream_test = hotshom.running(homeotest = homeotest, nb_digit=nb_test, train=False, LR=True)

2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_5000_[None, None]_homeo.pkl
../Records/nmnist/test/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_1000_[None, None]_homeo.pkl
no jitter


KeyboardInterrupt: 

In [4]:
# get indices for transitions from one digit to another 
def getdigind(stream):
    t = np.array(stream[2])
    newdig = [0]
    for i in range(len(t)-1):
        if t[i]>t[i+1]:
            newdig.append(i+1)
    newdig.append(i)
    return newdig

In [6]:
import numpy as np
events_train = np.zeros([len(stream_train[2]), 4])
ordering = 'xytp'
for i in range(4):
    events_train[:, i] = stream_train[i][:]

In [7]:
events_train.shape

(19523192, 4)

In [8]:
len(stream_train[4])

19523192

In [9]:
X_train = events_train.astype(int) #torch.tensor(events_train)
y_train = stream_train[4] # torch.tensor(stream_train[4])
digind_train = getdigind(stream_train) #torch.tensor(getdigind(stream_train))

In [5]:
import torch
from torch.utils.data import Dataset, TensorDataset
class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    sensor_size = [34,34]
    ordering = "xytp"
    
    def __init__(self, tensors, digind, transform=None):
        self.X_train, self.y_train = tensors
        assert (self.X_train.shape[0] == len(self.y_train))
        self.transform = transform
        self.digind = digind

    def __getitem__(self, index ):
        
        x = self.X_train[self.digind[index]:self.digind[index+1]]

        if self.transform:
            x = self.transform(x, self.sensor_size, self.ordering)

        y = self.y_train[self.digind[index]]
        
        return x, y

    def __len__(self):
        return len(self.digind)-1

In [11]:
# Dataset w/o any tranformations
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), digind=digind_train, transform=None)
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=1)

In [12]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape, y)
    break # we need just one batch

torch.Size([1, 4872, 4]) tensor([7])


In [13]:
from tonic.transforms import AERtoVector
train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), digind = digind_train, transform=AERtoVector())
train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=1)

In [14]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape, y)
    break # we need just one batch

torch.Size([1, 4872, 18496]) tensor([7])


# all in one function

In [18]:
def get_loader(name):

    hotshom, homeotest = netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter)
    stream_train = hotshom.running(homeotest = homeotest, nb_digit=nb_train, train=True, LR=True)
    stream_test = hotshom.running(homeotest = homeotest, nb_digit=nb_test, train=False, LR=True)

    # get indices for transitions from one digit to another 
    def getdigind(stream):
        t = np.array(stream[2])
        newdig = [0]
        for i in range(len(t)-1):
            if t[i]>t[i+1]:
                newdig.append(i+1)
        newdig.append(i)
        return newdig

    events_train = np.zeros([len(stream_train[2]), 4])
    ordering = 'xytp'
    for i in range(4):
        events_train[:, i] = stream_train[i][:]

    X_train = events_train.astype(int) #torch.tensor(events_train)
    y_train = stream_train[4] # torch.tensor(stream_train[4])
    digind_train = getdigind(stream_train) #torch.tensor(getdigind(stream_train))

    # Dataset w/o any tranformations
    train_dataset_normal = CustomTensorDataset(tensors=(X_train, y_train), digind=digind_train, transform=None)
    train_loader = torch.utils.data.DataLoader(train_dataset_normal, batch_size=1)
    return train_loader

train_loader = get_loader('homhots') 

2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_5000_[None, None]_homeo.pkl
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_1000_[None, None]_homeo.pkl


In [19]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape, y)
    break # we need just one batch

torch.Size([1, 4872, 4]) tensor([7])
